# Data Wrangling Section

In [1]:
import pandas as pd
import matplotlib as plt
import numpy as np
print('done')

done


First, I will merge all the weekly data into one dataframe.

In [2]:
df_w1 = pd.read_csv('Data/week1.csv')
df_w2 = pd.read_csv('Data/week2.csv')
df_w3 = pd.read_csv('Data/week3.csv')
df_w4 = pd.read_csv('Data/week4.csv')
df_w5 = pd.read_csv('Data/week5.csv')
df_w6 = pd.read_csv('Data/week6.csv')
df_w7 = pd.read_csv('Data/week7.csv')
df_w8 = pd.read_csv('Data/week8.csv')
print('done')

done


In [3]:
df_weekly = pd.concat([df_w1,df_w2, df_w3, df_w4, df_w5, df_w6, df_w7, df_w8], axis=0)
rows = df_w1.shape[0]+ df_w2.shape[0] + df_w3.shape[0] + df_w4.shape[0] + df_w5.shape[0] + df_w6.shape[0] + df_w7.shape[0] + df_w8.shape[0]
print('Weekly DF Shape: ', df_weekly.shape)
print('Count of rows: ', rows)

Weekly DF Shape:  (8314178, 16)
Count of rows:  8314178


I've merged all the weekly play by play game data and I've confirmed the shape of the dataframe is accurate. Now, I need to add data to the weekly dataframe.

Since my goal is to estimate the expected receving yards on a play, I need to know which players are eligible to receive the ball. Thankfully, the pffScoutingData dataset tracks this stat with the 'pff_role' column . I will use the 'nflId', 'gameId' and 'playid' to merge the 2 dataframes. I will drop the rest of the columns that I do not need to save memory.

In [4]:
df_pff = pd.read_csv('Data/pffScoutingData.csv')
print('done')

done


In [5]:
df_pff.drop(['pff_hit', 'pff_hurry', 'pff_sack', 'pff_beatenByDefender', 'pff_hitAllowed', 'pff_hurryAllowed', 'pff_sackAllowed', 'pff_nflIdBlockedPlayer', 'pff_blockType', 'pff_backFieldBlock'], inplace=True, axis=1)
df_pff.columns

Index(['gameId', 'playId', 'nflId', 'pff_role', 'pff_positionLinedUp'], dtype='object')

In the play by play tracking data, the football is tracked as NaN and since the pandas merge method will drop NaN values, I will create two separate dataframes one with the football tracking time and the other without the football tracking times. I will merge the pff dataframe with the non-football dataframe and then use the concat function to put the data back in to main dataframe.

In [6]:
df_football = df_weekly[df_weekly['team'] == 'football']

In [7]:
df_main = df_pff.merge(df_weekly[pd.notnull(df_weekly.nflId)], on=['gameId', 'playId', 'nflId'])
df_main.shape

(7952692, 18)

In [8]:
df_main = pd.concat([df_main,df_football])
df_main

,gameId,playId,nflId,pff_role,pff_positionLinedUp,frameId,time,jerseyNumber,team,playDirection,x,y,s,a,dis,o,dir,event
0,2021090900,97,25511.0,Pass,QB,1,2021-09-10T00:26:31.100,12.0,TB,right,37.77,24.22,0.29,0.30,0.03,165.16,84.99,None
1,2021090900,97,25511.0,Pass,QB,2,2021-09-10T00:26:31.200,12.0,TB,right,37.78,24.22,0.23,0.11,0.02,164.33,92.87,None
2,2021090900,97,25511.0,Pass,QB,3,2021-09-10T00:26:31.300,12.0,TB,right,37.78,24.24,0.16,0.10,0.01,160.24,68.55,None
3,2021090900,97,25511.0,Pass,QB,4,2021-09-10T00:26:31.400,12.0,TB,right,37.73,24.25,0.15,0.24,0.06,152.13,296.85,None
4,2021090900,97,25511.0,Pass,QB,5,2021-09-10T00:26:31.500,12.0,TB,right,37.69,24.26,0.25,0.18,0.04,148.33,287.55,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
978944,2021110100,4433,NaN,NaN,NaN,54,2021-11-02T03:20:26.500,NaN,football,right,24.17,27.89,2.47,2.28,0.25,NaN,NaN,None
978945,2021110100,4433,NaN,NaN,NaN,55,2021-11-02T03:20:26.600,NaN,football,right,24.36,28.03,2.36,2.16,0.24,NaN,NaN,None
978946,2021110100,4433,NaN,NaN,NaN,56,2021-11-02T03:20:26.700,NaN,football,right,24.55,28.17,2.25,1.45,0.23,NaN,NaN,None
978947,2021110100,4433,NaN,NaN,NaN,57,2021-11-02T03:20:26.800,NaN,football,right,24.73,28.31,2.28,0.72,0.23,NaN,NaN,None


The pff and weekly dataset have been merged. Now, the 'plays' dataset needs to be merged with this current dataset. 

In [9]:
df_plays = pd.read_csv('Data/plays.csv')
print(df_plays.shape)
df_plays['dropBackType'].value_counts()
df_plays = df_plays[['gameId', 'playId', 'passResult', 'penaltyYards', 'prePenaltyPlayResult', 'playResult', 'absoluteYardlineNumber', 'pff_playAction', 'pff_passCoverage', 'dropBackType']]
print(df_plays.shape)
df_plays.columns

(8557, 32)
(8557, 10)


Index(['gameId', 'playId', 'passResult', 'penaltyYards',
       'prePenaltyPlayResult', 'playResult', 'absoluteYardlineNumber',
       'pff_playAction', 'pff_passCoverage', 'dropBackType'],
      dtype='object')

In [10]:
df_main = df_main.merge(df_plays, on=['gameId', 'playId'])
df_main.shape

(8314178, 26)

I've added 8 new columns to the existing dataframe using 'gameId' and 'playId'. Now, I will clean up the main dataframe to get it ready for the EDA section.

In [11]:
df_main.dtypes

gameId                      int64
playId                      int64
nflId                     float64
pff_role                   object
pff_positionLinedUp        object
frameId                     int64
time                       object
jerseyNumber              float64
team                       object
playDirection              object
x                         float64
y                         float64
s                         float64
a                         float64
dis                       float64
o                         float64
dir                       float64
event                      object
passResult                 object
penaltyYards              float64
prePenaltyPlayResult        int64
playResult                  int64
absoluteYardlineNumber    float64
pff_playAction              int64
pff_passCoverage           object
dropBackType               object
dtype: object

Dropping the columns I do not need going forward.

In [12]:
df_main.drop(['jerseyNumber', 'time', 'playDirection', 'absoluteYardlineNumber'], inplace=True, axis=1)

In [13]:
# 2021092700 3905 72
df_main5 = df_main[df_main['gameId'] == 2021092700]
df_main5 = df_main5[df_main5['playId'] == 3905]
df_main5 = df_main5[df_main5['frameId'] == 72]
df_main5 = df_main5[['x','y','s','a','dis','o','dir']]
df_main5

,x,y,s,a,dis,o,dir
3279066,39.87,33.25,1.87,1.36,0.20,34.81,17.32
3279143,35.54,25.52,1.52,0.56,0.16,330.05,334.10
3279220,2.65,28.20,3.96,2.09,0.41,352.85,359.58
3279297,35.60,49.75,7.16,3.21,0.74,329.72,341.67
3279374,32.92,50.09,7.68,4.29,0.81,4.14,358.82
3279451,27.61,45.34,5.87,2.61,0.62,1.04,352.86
3279528,19.85,44.88,5.42,1.36,0.55,355.91,346.84
3279605,10.01,53.31,1.92,2.22,0.19,90.04,267.26
3279682,2.37,27.93,4.76,1.56,0.48,81.54,350.97
3279759,9.47,47.53,2.57,0.38,0.26,83.20,301.55


I will check the x,y,s,a,dis,o,dir columns to make sure there are no values that do not fit their respective criteria

In [14]:
# Method to check the min and max number values of each column
def find_min_max(col, df):
    sub_arr = df[col].unique()
    unique, counts = np.unique(sub_arr, return_counts=True)
    unique_min = min(unique)
    unique_max = max(unique)
    print('min: ', unique_min)
    print('max: ', unique_max)

In [15]:
df_main = df_main[(df_main['x'] >= 0) & (df_main['x'] <= 120)]
print(df_main.shape)
find_min_max('x', df_main)

(8314066, 22)
min:  0.0
max:  120.0


In [16]:
df_main = df_main[(df_main['y'] >= 0) & (df_main['y'] <= 53.3)]
find_min_max('y', df_main)

min:  0.0
max:  53.3


In [17]:
find_min_max('o', df_main)

min:  0.0
max:  360.0


In [18]:
find_min_max('dir', df_main)

min:  0.0
max:  360.0


In [19]:
sub_arr = df_main['s'].unique()
unique, counts = np.unique(sub_arr, return_counts=True)
unique_min = min(unique)
print('min: ', unique_min)

min:  0.0


In [20]:
sub_arr = df_main['a'].unique()
unique, counts = np.unique(sub_arr, return_counts=True)
unique_min = min(unique)
print('min: ', unique_min)

min:  0.0


In [21]:
sub_arr = df_main['dis'].unique()
unique, counts = np.unique(sub_arr, return_counts=True)
unique_min = min(unique)
print('min: ', unique_min)

min:  0.0


I've confirmed that x,y,s,a,dis,o,dir columns all have accurate data now, I've dropped rows in columns that do not fit the criteria laid out by the NFL that came with the dataset. I will investigate the rest of the columns in the dataset to confirm they fit the criteria as well.

In [22]:
print('FrameID: ', find_min_max('frameId', df_main))
print('prePenaltyPlayResult: ', find_min_max('prePenaltyPlayResult', df_main))
print('penaltyYards: ', find_min_max('penaltyYards', df_main))
print('playResult: ', find_min_max('playResult', df_main))
print('pff_playAction: ', find_min_max('pff_playAction', df_main))

min:  1
max:  203
FrameID:  None
min:  -34
max:  91
prePenaltyPlayResult:  None
min:  -18.0
max:  50.0
penaltyYards:  None
min:  -34
max:  91
playResult:  None
min:  0
max:  1
pff_playAction:  None


In [23]:
count = 0
count_instances = 0
play_list = []
game_list = []
for game in df_main['gameId'].unique():
    df = df_main[df_main['gameId'] == game]
    unique_plays = df['playId'].unique()
    for play in unique_plays:
        df1 = df[df['playId'] == play]
        frames = df1['frameId'].unique()
        for frame in frames:
            df2 = df1[df1['frameId'] == frame]
            if df2.shape[0] == 23:
                pass
            else:
                count_instances += df2.shape[0]
                if (game in game_list) &( play in play_list):
                    pass
                else:
                    play_list.append(play)
                    game_list.append(game)
                    count += 1
                    
print(count)
print(count_instances)

94
19159


All the min and max units above are accurate.

In [24]:
df_main = df_main.drop(df_main[(df_main['gameId'].isin(game_list)) & (df_main['playId'].isin(play_list))].index)
df_main.shape

(8051817, 22)

I will confirm that each frame has 23 "players" or nflIds. 11 players on each team and an extra row for the football.

In [25]:
count = 0
count_instances = 0
play_list = []
game_list = []
for game in df_main['gameId'].unique():
    df = df_main[df_main['gameId'] == game]
    unique_plays = df['playId'].unique()
    for play in unique_plays:
        df1 = df[df['playId'] == play]
        frames = df1['frameId'].unique()
        for frame in frames:
            df2 = df1[df1['frameId'] == frame]
            if df2.shape[0] == 23:
                pass
            else:
                count_instances += df2.shape[0]
                if (game in game_list) &( play in play_list):
                    pass
                else:
                    play_list.append(play)
                    game_list.append(game)
                    count += 1
                    
print(count)
print(count_instances)

0
0


The dataframe now has accurate player numbers for each frame and play. Now is the time to deal with NaN values.

In [27]:
df_main.isna().sum()

gameId                        0
playId                        0
nflId                    350079
pff_role                 350079
pff_positionLinedUp      350079
frameId                       0
team                          0
x                             0
y                             0
s                             0
a                             0
dis                           0
o                        350079
dir                      350079
event                         0
passResult                    0
penaltyYards            7271634
prePenaltyPlayResult          0
playResult                    0
pff_playAction                0
pff_passCoverage              0
dropBackType             529092
dtype: int64

According to the NFL, the tracking of 'football' doesn't include nflId, pff_role, pff_positionLinedUp, o, dir. For o and dir, I will 0 out the NaNs. NflId, pff_role, and pff_positionLinedUp will be changed in the pre-processing step of the process. I will need the information for the exploratory data analysis step. 

First, I am going to confirm that the NaNs in the dataset are all related to the 'football' tracking rows, and then 0 out the fields.

In [28]:
df_main[df_main['team'] == 'football'].isna().sum()

gameId                       0
playId                       0
nflId                   350079
pff_role                350079
pff_positionLinedUp     350079
frameId                      0
team                         0
x                            0
y                            0
s                            0
a                            0
dis                          0
o                       350079
dir                     350079
event                        0
passResult                   0
penaltyYards            316158
prePenaltyPlayResult         0
playResult                   0
pff_playAction               0
pff_passCoverage             0
dropBackType             23004
dtype: int64

In [29]:
df_main[["o","dir"]] = df_main[["o","dir"]].fillna(0)
df_main.isna().sum()

gameId                        0
playId                        0
nflId                    350079
pff_role                 350079
pff_positionLinedUp      350079
frameId                       0
team                          0
x                             0
y                             0
s                             0
a                             0
dis                           0
o                             0
dir                           0
event                         0
passResult                    0
penaltyYards            7271634
prePenaltyPlayResult          0
playResult                    0
pff_playAction                0
pff_passCoverage              0
dropBackType             529092
dtype: int64

The penalty yards row is needed for further analysis in the future. But when there is no penalty on a play, it is tracked as NaN, I will replace them with 0s since there was 0 penalty yardage on the play.

In [30]:
df_main['penaltyYards'] = df_main['penaltyYards'].fillna(0)
df_main.isna().sum()

gameId                       0
playId                       0
nflId                   350079
pff_role                350079
pff_positionLinedUp     350079
frameId                      0
team                         0
x                            0
y                            0
s                            0
a                            0
dis                          0
o                            0
dir                          0
event                        0
passResult                   0
penaltyYards                 0
prePenaltyPlayResult         0
playResult                   0
pff_playAction               0
pff_passCoverage             0
dropBackType            529092
dtype: int64

In [31]:
df_main['dropBackType'].value_counts()

TRADITIONAL               5798668
SCRAMBLE                  1136660
DESIGNED_ROLLOUT_RIGHT     266685
DESIGNED_ROLLOUT_LEFT      147361
SCRAMBLE_ROLLOUT_RIGHT     141703
SCRAMBLE_ROLLOUT_LEFT       25921
DESIGNED_RUN                 4439
UNKNOWN                      1288
Name: dropBackType, dtype: int64

The dropBackType field is only needed to drop all plays that were 'designed_run's. Designed Runs do not effect an analysis on passing plays. I will drop all fields where the play is a designed run, and then make sure that each frame/play has 23 participants.

In [32]:
df_main.shape

(8051817, 22)

In [34]:
df_main = df_main[df_main['dropBackType'] != 'DESIGNED_RUN']
df_main.shape

(8047378, 22)

In [35]:
count = 0
count_instances = 0
play_list = []
game_list = []
for game in df_main['gameId'].unique():
    df = df_main[df_main['gameId'] == game]
    unique_plays = df['playId'].unique()
    for play in unique_plays:
        df1 = df[df['playId'] == play]
        frames = df1['frameId'].unique()
        for frame in frames:
            df2 = df1[df1['frameId'] == frame]
            if df2.shape[0] == 23:
                pass
            else:
                count_instances += df2.shape[0]
                if (game in game_list) &( play in play_list):
                    pass
                else:
                    play_list.append(play)
                    game_list.append(game)
                    count += 1
                    
print(count)
print(count_instances)

0
0


In [39]:
df_main = df_main.drop(['dropBackType'], axis=1)

df_main.isna().sum()

gameId                       0
playId                       0
nflId                   349886
pff_role                349886
pff_positionLinedUp     349886
frameId                      0
team                         0
x                            0
y                            0
s                            0
a                            0
dis                          0
o                            0
dir                          0
event                        0
passResult                   0
penaltyYards                 0
prePenaltyPlayResult         0
playResult                   0
pff_playAction               0
pff_passCoverage             0
dtype: int64

In [41]:
df_main.head()

,gameId,playId,nflId,pff_role,pff_positionLinedUp,frameId,team,x,y,s,...,dis,o,dir,event,passResult,penaltyYards,prePenaltyPlayResult,playResult,pff_playAction,pff_passCoverage
0,2021090900,97,25511.0,Pass,QB,1,TB,37.77,24.22,0.29,...,0.03,165.16,84.99,None,I,0.0,0,0,0,Cover-1
1,2021090900,97,25511.0,Pass,QB,2,TB,37.78,24.22,0.23,...,0.02,164.33,92.87,None,I,0.0,0,0,0,Cover-1
2,2021090900,97,25511.0,Pass,QB,3,TB,37.78,24.24,0.16,...,0.01,160.24,68.55,None,I,0.0,0,0,0,Cover-1
3,2021090900,97,25511.0,Pass,QB,4,TB,37.73,24.25,0.15,...,0.06,152.13,296.85,None,I,0.0,0,0,0,Cover-1
4,2021090900,97,25511.0,Pass,QB,5,TB,37.69,24.26,0.25,...,0.04,148.33,287.55,None,I,0.0,0,0,0,Cover-1


This data set is now ready for the exploratory data analysis phase. I have dealt with missing values, consolidated the required for my analysis, and checked it's quality.

In [ ]:
df_main.to_csv('Data/cleaned_data', index=False)